In [1]:
import os
import random
import json
import time
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Only logs errors

nltk.download('all')

2024-12-03 21:52:33.961589: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 21:52:34.047804: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 21:52:34.150328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733237554.227900    5739 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733237554.251848    5739 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 21:52:34.448432: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

True

In [2]:
lemmatizer = WordNetLemmatizer()
work_dir = os.path.dirname(os.path.abspath('.'))

In [13]:
intents = json.loads(open(os.path.join(work_dir, 'data/intents.json')).read())

words = []
classes = []
documents = []
ignores = ["?", "!", ".", ","]

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent["tag"]))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

words = sorted(set(lemmatizer.lemmatize(word.lower()) for word in words if word not in ignores))
classes = sorted(set(classes))

pickle.dump(words, open(os.path.join(work_dir, "model/words.pkl"), "wb"))
pickle.dump(classes, open(os.path.join(work_dir, "model/classes.pkl"), "wb"))

In [14]:
training = []
output_empty = [0] * len(classes)

for document in documents:
  word_bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

  for word in words:
    word_bag.append(1) if word in word_patterns else word_bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([word_bag, output_row])
  
print(len(training))  # Total number of samples
print(len(training[0][0]), len(words))  # Length of each word bag (should be len(words))
print(len(training[0][1]), len(classes))  # Length of each output row (should be len(classes))

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

81
107 107
20 20


In [15]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) # First layer
model.add(Dropout(0.5)) # Randomly disable 50% of neurons
model.add(Dense(64, activation='relu')) # Second layer
model.add(Dropout(0.5)) # Randomly disable 50% of neurons
model.add(Dense(len(train_y[0]), activation='softmax')) # Output layer

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Stochastic Gradient Descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save(os.path.join(work_dir, "model/chatbot.h5"))

Epoch 1/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1165 - loss: 3.0211      
Epoch 2/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0654 - loss: 2.9772     
Epoch 3/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0620 - loss: 2.8492     
Epoch 4/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1889 - loss: 2.8228     
Epoch 5/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1313 - loss: 2.8414     
Epoch 6/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1741 - loss: 2.6707     
Epoch 7/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1561 - loss: 2.6304     
Epoch 8/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2521 - loss: 2.4373 
Epoch 9/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2780 - loss: 2.3954 
Epoch 10/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3243 - loss: 2.2379 
Epoch 11/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3191 - loss: 2.2098     
Epoch 12/200
17/17 